In [60]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# ***`Libraries`***

In [61]:
import pandas as pd
import re
import json
import ast
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx


# ***`Read Data`***

In [62]:
file_path = '/content/drive/My Drive/Colab Notebooks/SocialNetworkingCourse/Project VNexpress/vnexpress_articles_4.xlsx'
df = pd.read_excel(file_path)

In [63]:
def clean_date(date_str):
    # Check if date_str is a string before splitting
    if isinstance(date_str, str):
        # Lấy phần ngày tháng năm (loại bỏ các ký tự không cần thiết)
        date_part = date_str.split(",")[1].strip()
        # Chuyển đổi chuỗi thành định dạng datetime
        date_obj = pd.to_datetime(date_part, format='%d/%m/%Y')
        # Định dạng lại chuỗi ngày
        return date_obj.strftime('%A, %d/%m/%Y')
    else:
        # Handle non-string values (e.g., NaN) by returning a default value or skipping
        return pd.NaT  # Return NaT (Not a Time) for invalid dates

df['Formatted_Date'] = df['Date'].apply(clean_date)

In [64]:
df = df.drop(columns=['Date', 'Detailed Title', 'Comments Count', 'Content', 'Link'])
df.head(5)

,Title,Author,Author Link,Category,Description,Comments,Formatted_Date
0,Tam giác mục tiêu 2025,Cấn Văn Lực,https://vnexpress.net/tac-gia/can-van-luc-998....,Kinh doanh & quản trị,Chuyên gia kinh tế,"[{'Nickname': 'dinhquang3082003', 'Nickname Li...","Wednesday, 01/01/2025"
1,Động lực kinh tế tư nhân,Nguyễn Hoa Cương,https://vnexpress.net/tac-gia/nguyen-hoa-cuong...,Kinh doanh & quản trị,Chuyên gia kinh tế,"[{'Nickname': 'Nhật Nhật Tân.', 'Nickname Link...","Wednesday, 01/01/2025"
2,Miễn học phí ngành Y?,Nguyễn Minh Hoàng,https://vnexpress.net/tac-gia/nguyen-minh-hoan...,Giáo dục & tri thức,Nhà công tác xã hội và phát triển cộng đồng,"[{'Nickname': 'Khánh Super', 'Nickname Link': ...","Tuesday, 31/12/2024"
3,Đến Việt Nam chữa bệnh,Nguyễn Hồng Hà,https://vnexpress.net/tac-gia/nguyen-hong-ha-1...,Y tế & sức khỏe,Bác sĩ phẫu thuật,"[{'Nickname': 'hanhantramchuong', 'Nickname Li...","Monday, 30/12/2024"
4,'Ăn cắp' thời công nghệ,Quan Thế Dân,https://vnexpress.net/tac-gia/quan-the-dan-144...,Văn hóa & lối sống,"Bác sĩ, Tiến sĩ Y học","[{'Nickname': 'thecong85', 'Nickname Link': 'h...","Sunday, 29/12/2024"


In [65]:
# Function to process Comments_Post
def extract_comments_info(comments):
    result = []
    # Check if 'comments' is a string and attempt to convert to a list of dictionaries
    if isinstance(comments, str):
        try:
            comments = ast.literal_eval(comments)  # Safely evaluate the string
        except (SyntaxError, ValueError):
            # Handle cases where conversion fails (e.g., invalid JSON)
            return result  # Return empty result if conversion fails
    #Check if comments is not a list and also not a string before proceeding
    elif not isinstance(comments, list):
        return result #If comments is neither a string nor a list (e.g a float) return empty list

    # Now proceed with the original logic if 'comments' is a list of dictionaries
    for comment in comments:
        if isinstance(comment, dict):  # Ensure comment is a dictionary
            result.append({
                "Nickname": comment.get("Nickname", ""),
                "Nickname Link": comment.get("Nickname Link", ""),
                "Reply Nicknames": comment.get("Reply Nicknames", []),
                "Reply Links": comment.get("Reply Links", [])
            })
    return result

# Apply extraction to the Comments_Post column
df['Processed_Comments'] = df['Comments'].apply(extract_comments_info)

# Drop the original Comments_Post column for cleaner display
df.drop(columns=['Comments'], inplace=True)

In [66]:
# Extract Author ID from Author Link (keep full ID including numbers)
df["Author Link"] = df["Author Link"].str.extract(r".*/(.*)\.html")

# Extract Processed_Comments into structured form
def process_comments(comments):
    processed = []
    for comment in comments:
        nickname_link = comment.get("Nickname Link", "")
        reply_links = comment.get("Reply Links", [])
        # Extract IDs
        nickname_id = nickname_link.split("/")[-1] if nickname_link else ""
        reply_ids = [link.split("/")[-1] for link in reply_links]
        processed.append({
            "Nickname ID": nickname_id,
            "Reply IDs": reply_ids,
        })
    return processed

df["Processed_Comments"] = df["Processed_Comments"].apply(process_comments)


df = df.drop(columns = ['Title', 'Author', "Formatted_Date"])
# Display the result
df.head(2)

,Author Link,Category,Description,Processed_Comments
0,can-van-luc-998,Kinh doanh & quản trị,Chuyên gia kinh tế,"[{'Nickname ID': '1080039082', 'Reply IDs': []..."
1,nguyen-hoa-cuong-1336,Kinh doanh & quản trị,Chuyên gia kinh tế,"[{'Nickname ID': '1002685738', 'Reply IDs': []}]"


In [67]:
# Sử dụng phương pháp thay thế để đảm bảo trích xuất chính xác
# Tách số ID từ URL
df['Author ID'] = df['Author Link'].astype(str).apply(lambda x: x.split('-')[-1].replace('.html', '') if '-' in x else x)
df = df.drop(columns=['Author Link'])
df

,Category,Description,Processed_Comments,Author ID
0,Kinh doanh & quản trị,Chuyên gia kinh tế,"[{'Nickname ID': '1080039082', 'Reply IDs': []...",998
1,Kinh doanh & quản trị,Chuyên gia kinh tế,"[{'Nickname ID': '1002685738', 'Reply IDs': []}]",1336
2,Giáo dục & tri thức,Nhà công tác xã hội và phát triển cộng đồng,"[{'Nickname ID': '1026010466', 'Reply IDs': ['...",1955
3,Y tế & sức khỏe,Bác sĩ phẫu thuật,"[{'Nickname ID': '1051012051', 'Reply IDs': ['...",1279
4,Văn hóa & lối sống,"Bác sĩ, Tiến sĩ Y học","[{'Nickname ID': '1059731000', 'Reply IDs': ['...",1447
...,...,...,...,...
3260,NaN,NaN,[],nan
3261,Phạm Hồng Phước,Nhà báo,"[{'Nickname ID': 'javascript:;', 'Reply IDs': ...",30
3262,Võ Xuân Sơn,Bác sĩ,[],59
3263,NaN,NaN,"[{'Nickname ID': 'javascript:;', 'Reply IDs': ...",nan


In [69]:
G = nx.DiGraph()

In [70]:
# Lặp qua từng hàng trong DataFrame
for idx, row in df.iterrows():
    # Thêm tác giả vào đồ thị
    author_id = row["Author ID"]
    category = row["Category"]
    description = row["Description"]
    G.add_node(author_id, role="author", category=category, description=description)

    # Thêm các người bình luận và kết nối họ với tác giả
    for comment in row["Processed_Comments"]:  # Duyệt qua danh sách bình luận
        nickname_id = comment["Nickname ID"]
        replies = comment["Reply IDs"]

        # Thêm người bình luận với thuộc tính
        G.add_node(nickname_id, role="commenter")
        G.add_edge(nickname_id, author_id, relationship="commented_on")

        # Thêm người phản hồi và kết nối họ với người bình luận
        for reply_id in replies:
            G.add_node(reply_id, role="replier")
            G.add_edge(reply_id, nickname_id, relationship="replied_to")


In [71]:
num_nodes = G.number_of_nodes()
num_edges = G.number_of_edges()
print(f"Number of Nodes: {num_nodes}")
print(f"Number of Edges: {num_edges}")

Number of Nodes: 41547
Number of Edges: 95393


In [72]:
# Vẽ đồ thị với kích thước và các thông số tối ưu hóa
plt.figure(figsize=(16, 12))  # Kích thước đồ thị lớn hơn

# Điều chỉnh layout với tham số k nhỏ hơn để giảm độ chồng chéo của các node
pos = nx.spring_layout(G, seed=42, k=0.05, iterations=50)  # Giảm k để nới rộng các node

# Lấy mối quan hệ giữa các node
edge_labels = nx.get_edge_attributes(G, "relationship")

# Vẽ đồ thị
nx.draw(
    G,
    pos,
    with_labels=False,  # Không hiển thị nhãn cho tất cả nodes để tránh quá tải
    node_size=50,  # Giảm kích thước nodes
    font_size=8,  # Giảm kích thước font chữ
    font_color="black",
    edge_color="gray",
    width=0.5,  # Giảm độ dày của các cạnh
    alpha=0.5,  # Độ mờ của các cạnh
    node_color="lightblue",
    edgecolors="black"
)

# Hiển thị tên mối quan hệ trên các cạnh (nếu cần)
nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, font_color="red", font_size=6)

# Tiêu đề đồ thị
plt.title("Social Network Graph - VnExpress", fontsize=16)
plt.show()


Output hidden; open in https://colab.research.google.com to view.

In [74]:
# Export the graph to a file in GraphML format
output_file = "/content/drive/My Drive/Colab Notebooks/SocialNetworkingCourse/Project VNexpress/social_network.graphml"
nx.write_graphml(G, output_file)

output_file


'/content/drive/My Drive/Colab Notebooks/SocialNetworkingCourse/Project VNexpress/social_network.graphml'